In [1]:
# data from http://grouplens.org/datasets/movielens/

In [5]:
import os
data_folder = os.path.join(os.path.expanduser("~"), "Data", "ml-100k")
print(data_folder)
ratings_filename = os.path.join(data_folder, "u.data")

/Users/yt/Data/ml-100k


In [6]:
import pandas as pd
pd.DataFrame.isin?

In [7]:
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],unit='s')
all_ratings[:5]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [8]:
L = [1,2,4,3]
L.sort(reverse = True)
print(L)

[4, 3, 2, 1]


In [9]:
# As you can see, there are no review for most movies, such as #213
all_ratings[all_ratings["UserID"] == 675].sort_values("MovieID").head(10)
#L = sorted(all_ratings["MovieID"])
#all_ratings["MovieID"].sort()
#all_ratings[:6]
#L[:6]
#all_ratings.sort()

,UserID,MovieID,Rating,Datetime
81098,675,86,4,1998-03-10 00:26:14
90696,675,223,1,1998-03-10 00:35:51
92650,675,235,1,1998-03-10 00:35:51
95459,675,242,4,1998-03-10 00:08:42
82845,675,244,3,1998-03-10 00:29:35
53293,675,258,3,1998-03-10 00:11:19
97286,675,269,5,1998-03-10 00:08:07
93720,675,272,3,1998-03-10 00:07:11
73389,675,286,4,1998-03-10 00:07:11
77524,675,303,5,1998-03-10 00:08:42


In [10]:
# Not all reviews are favourable! Our goal is "other recommended books", so we only want favourable reviews
all_ratings["Favorable"] = all_ratings["Rating"] > 3
all_ratings[10:15]

,UserID,MovieID,Rating,Datetime,Favorable
10,62,257,2,1997-11-12 22:07:14,False
11,286,1014,5,1997-11-17 15:38:45,True
12,200,222,5,1997-10-05 09:05:40,True
13,210,40,3,1998-03-27 21:59:54,False
14,224,29,3,1998-02-21 23:40:57,False


In [11]:
all_ratings[all_ratings["UserID"] == 0][:5]
#all_ratings[all_ratings["UserID"].isin(range(2))]

,UserID,MovieID,Rating,Datetime,Favorable


In [12]:
# Sample the dataset. You can try increasing the size of the sample, but the run time will be considerably longer
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]  # & ratings["UserID"].isin(range(100))]
ratings[:7]
#.isin(range(200))
#print(ratings)

,UserID,MovieID,Rating,Datetime,Favorable
0,196,242,3,1997-12-04 15:55:49,False
1,186,302,3,1998-04-04 19:22:22,False
2,22,377,1,1997-11-07 07:18:36,False
4,166,346,1,1998-02-02 05:33:16,False
6,115,265,2,1997-12-03 17:51:28,False
9,6,86,3,1997-12-31 21:16:53,False
10,62,257,2,1997-11-12 22:07:14,False


In [13]:
# We start by creating a dataset of each user's favourable reviews
favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings[-5:]

,UserID,MovieID,Rating,Datetime,Favorable
99848,5,174,5,1997-09-30 16:15:30,True
99950,130,93,5,1997-09-22 18:41:05,True
99951,130,121,5,1997-10-07 18:59:06,True
99959,193,690,4,1998-03-05 18:40:21,True
99978,113,975,5,1997-10-04 03:40:24,True


In [14]:
# We are only interested in the reviewers who have more than one review
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])
print(favorable_reviews_by_users[1])

frozenset({1, 3, 6, 7, 9, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23, 25, 28, 32, 33, 39, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 64, 65, 66, 68, 72, 75, 76, 77, 79, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 93, 95, 96, 98, 100, 106, 107, 108, 109, 111, 113, 114, 115, 119, 121, 123, 124, 127, 128, 129, 132, 133, 134, 135, 137, 144, 146, 150, 151, 152, 154, 156, 157, 160, 161, 162, 163, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 202, 203, 204, 206, 207, 208, 209, 210, 212, 214, 216, 221, 222, 223, 224, 227, 228, 229, 230, 234, 235, 236, 238, 239, 241, 242, 246, 248, 249, 250, 251, 253, 256, 257, 258, 265, 267, 268, 269, 270})


In [15]:
# Find out how many movies have favourable ratings
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie.sort_values("Favorable", ascending=False)[:5]
#sorted(num_favorable_by_movie["Favorable"],reverse=False)

,Favorable
MovieID,
50,100.0
100,89.0
258,83.0
181,79.0
174,74.0


In [16]:
from collections import defaultdict
#pd.DataFrame.items?
#pd.Series.str.len?
#sorted?
#len?
#print(len([1,2]))
#print(len(favorable_reviews_by_users.items()))
#pd.read_csv?
#def?
#__import__?
#filter?

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])
frozenset?
F = {}


#for user,reviews in favorable_reviews_by_users.items():
#    print(user,reviews)
frequent_itemsets[1] = dict((frozenset([movie_id]), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)
for itemset in frequent_itemsets[1]:
    print (itemset)

NameError: name 'min_support' is not defined

In [17]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset([movie_id]), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)


print(frequent_itemsets)
print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

{1: {frozenset({1}): 66.0, frozenset({7}): 67.0, frozenset({9}): 53.0, frozenset({50}): 100.0, frozenset({56}): 67.0, frozenset({64}): 58.0, frozenset({79}): 58.0, frozenset({98}): 70.0, frozenset({100}): 89.0, frozenset({127}): 70.0, frozenset({172}): 59.0, frozenset({174}): 74.0, frozenset({181}): 79.0, frozenset({258}): 83.0, frozenset({286}): 59.0, frozenset({313}): 60.0}}
There are 16 movies with more than 50 favorable reviews
I found 93 frequent itemsets of length 2
I found 295 frequent itemsets of length 3
I found 593 frequent itemsets of length 4
I found 785 frequent itemsets of length 5
I found 677 frequent itemsets of length 6
I found 373 frequent itemsets of length 7
I found 126 frequent itemsets of length 8
I found 24 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


In [18]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

Found a total of 2968 frequent itemsets


In [19]:
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

There are 15285 candidate rules


In [20]:
print(candidate_rules[:100])

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56), (frozenset({56}), 1), (frozenset({1}), 64), (frozenset({64}), 1), (frozenset({79}), 1), (frozenset({1}), 79), (frozenset({98}), 1), (frozenset({1}), 98), (frozenset({100}), 1), (frozenset({1}), 100), (frozenset({127}), 1), (frozenset({1}), 127), (frozenset({172}), 1), (frozenset({1}), 172), (frozenset({174}), 1), (frozenset({1}), 174), (frozenset({181}), 1), (frozenset({1}), 181), (frozenset({7}), 9), (frozenset({9}), 7), (frozenset({7}), 50), (frozenset({50}), 7), (frozenset({7}), 56), (frozenset({56}), 7), (frozenset({7}), 64), (frozenset({64}), 7), (frozenset({7}), 79), (frozenset({79}), 7), (frozenset({7}), 98), (frozenset({98}), 7), (frozenset({7}), 100), (frozenset({100}), 7), (frozenset({7}), 127), (frozenset({127}), 7), (frozenset({7}), 172), (frozenset({172}), 7), (frozenset({7}), 174), (frozenset({174}), 7), (frozenset({7}), 181), (frozenset({181}), 7), (frozenset({7})

In [21]:
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}
print(rule_confidence)

{(frozenset({7}), 1): 0.4626865671641791, (frozenset({1}), 7): 0.4696969696969697, (frozenset({50}), 1): 0.5, (frozenset({1}), 50): 0.7575757575757576, (frozenset({1}), 56): 0.48484848484848486, (frozenset({56}), 1): 0.47761194029850745, (frozenset({1}), 64): 0.45454545454545453, (frozenset({64}), 1): 0.5172413793103449, (frozenset({79}), 1): 0.5344827586206896, (frozenset({1}), 79): 0.4696969696969697, (frozenset({98}), 1): 0.5142857142857142, (frozenset({1}), 98): 0.5454545454545454, (frozenset({100}), 1): 0.47191011235955055, (frozenset({1}), 100): 0.6363636363636364, (frozenset({127}), 1): 0.4714285714285714, (frozenset({1}), 127): 0.5, (frozenset({172}), 1): 0.5084745762711864, (frozenset({1}), 172): 0.45454545454545453, (frozenset({174}), 1): 0.5540540540540541, (frozenset({1}), 174): 0.6212121212121212, (frozenset({181}), 1): 0.4810126582278481, (frozenset({1}), 181): 0.5757575757575758, (frozenset({7}), 9): 0.373134328358209, (frozenset({9}), 7): 0.4716981132075472, (frozenset(

In [22]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [23]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))
sorted?

5152


In [24]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [35]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({98, 181}) they will also recommend 50
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset({172, 79}) they will also recommend 174
 - Confidence: 1.000

Rule #3
Rule: If a person recommends frozenset({258, 172}) they will also recommend 174
 - Confidence: 1.000

Rule #4
Rule: If a person recommends frozenset({1, 181, 7}) they will also recommend 50
 - Confidence: 1.000

Rule #5
Rule: If a person recommends frozenset({1, 172, 7}) they will also recommend 174
 - Confidence: 1.000



In [26]:
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join(data_folder, "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [27]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

In [39]:
get_movie_name({1,2,3})

IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 1.000

Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Confidence: 1.000

Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000



In [34]:
# Evaluation using test data
test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
print(test_dataset[:10])
test_favorable = test_dataset[test_dataset["Favorable"]]
#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])
#test_not_favourable_by_users = dict((k, frozenset(v.values)) for k, v in test_not_favourable.groupby("UserID")["MovieID"])
#test_users = test_dataset["UserID"].unique()

    UserID  MovieID  Rating            Datetime  Favorable
3      244       51       2 1997-11-27 05:02:03      False
5      298      474       4 1998-01-07 14:20:06       True
7      253      465       5 1998-04-03 18:34:27       True
8      305      451       3 1998-02-01 09:20:17      False
11     286     1014       5 1997-11-17 15:38:45       True
12     200      222       5 1997-10-05 09:05:40       True
13     210       40       3 1998-03-27 21:59:54      False
14     224       29       3 1998-02-21 23:40:57      False
15     303      785       3 1997-11-14 05:28:38      False
18     291     1042       4 1997-09-21 09:42:24       True


In [27]:
test_dataset[:5]

,UserID,MovieID,Rating,Datetime,Favorable
3,244,51,2,1997-11-27 05:02:03,False
5,298,474,4,1998-01-07 14:20:06,True
7,253,465,5,1998-04-03 18:34:27,True
8,305,451,3,1998-02-01 09:20:17,False
11,286,1014,5,1997-11-17 15:38:45,True


In [28]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [29]:
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

5152


In [30]:
sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

[((frozenset({64, 1, 7, 172, 79, 50}), 174), 1.0), ((frozenset({64, 258, 98, 7, 174, 181}), 172), 1.0), ((frozenset({64, 1, 98, 7, 79, 181, 56}), 174), 1.0), ((frozenset({64, 1, 98, 7, 172, 79, 181}), 174), 1.0), ((frozenset({64, 258, 98, 7, 174, 50, 181}), 172), 1.0)]


In [31]:
for index in range(10):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Shawshank Redemption, The (1994), Pulp Fiction (1994), Silence of the Lambs, The (1991), Star Wars (1977), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.909

Rule #2
Rule: If a person recommends Silence of the Lambs, The (1991), Fargo (1996), Empire Strikes Back, The (1980), Fugitive, The (1993), Star Wars (1977), Pulp Fiction (1994) they will also recommend Twelve Monkeys (1995)
 - Train Confidence: 1.000
 - Test Confidence: 0.609

Rule #3
Rule: If a person recommends Silence of the Lambs, The (1991), Empire Strikes Back, The (1980), Return of the Jedi (1983), Raiders of the Lost Ark (1981), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.946

Rule #4
Rule: If a person recommends Shawshank Redemption, The (1994), Silence of the Lambs, The (1991), Fargo (1996), Twelve Monkeys (1995), Empire Strikes Back, T